# Colab notebook for pytorch-lightning and time shift module

In [ ]:
%pip install wandb -qqq
import wandb
wandb.login()

In [ ]:
%cd /content/
!rm -r WorkoutDetector
!git clone --recursive https://github.com/iucario/WorkoutDetector.git -q

%cd WorkoutDetector
!git checkout colab
%pip install -e .

In [ ]:
%pip install einops -q
%pip install fvcore -q
%pip install timm -q
%pip install onnx onnxruntime-gpu -q
%pip install pytorch_lightning -q

Install mmaction2 just to supress import error.
It's not used in this notebook.

In [ ]:
%pip install git+https://github.com/open-mmlab/mim.git -q
!mim install mmcv-full==1.5.0 > /dev/null
%pip install git+https://github.com/open-mmlab/mmaction2 

In [ ]:
import base64

def parse_onedrive(link: str) -> str:
    """Parse onedrive link to download link.

    Args:
        link: str, start with `https://1drv.ms/u/s!`

    Returns:
        str, download link.
    """
    assert link.startswith('https://1drv.ms/u/s!')
    b = base64.urlsafe_b64encode(link.strip().encode('ascii'))
    s = b.decode('ascii')
    res = f'https://api.onedrive.com/v1.0/shares/u!{s}/root/content'
    return res

URL_VIDEO = parse_onedrive('https://1drv.ms/u/s!AiohV3HRf-34ipk0i1y2P1txpKYXFw')
URL_ANNO = parse_onedrive('https://1drv.ms/u/s!AiohV3HRf-34i_YvMob5Vpgvxjc3mQ')


Download video dataset. But it's not used. Raw frames dataset is used.

In [ ]:
# !wget $URL_VIDEO -O /content/videos.zip
# !wget $URL_ANNO -O /content/annotation.csv
# !zip -FF videos.zip --out video.zip > /dev/null
# !unzip video.zip > /dev/null

In [ ]:
import os
ROOT = '/content/WorkoutDetector'
from workoutdetector.datasets import RepcountDataset
data_root = os.path.join(ROOT, 'data')
repcount = RepcountDataset(root=data_root, split='train', download=True)

In [ ]:
from workoutdetector.settings import PROJ_ROOT
from workoutdetector.scripts.build_label_list import build_with_start

print('project root:', PROJ_ROOT)
data_root = os.path.join(PROJ_ROOT, 'data')
dst_dir = os.path.join(data_root, 'Binary')
build_with_start(data_root, dst_dir)
print('save label list to', dst_dir)

## Train

In [ ]:
!wget https://hanlab.mit.edu/projects/tsm/models/TSM_somethingv2_RGB_resnet50_shift8_blockres_avg_segment8_e45.pth \
    -P /content/WorkoutDetector/checkpoints/

In [ ]:
!python /content/WorkoutDetector/workoutdetector/trainer.py